In [ ]:
%pip install -U git+https://github.com/szagoruyko/pytorchviz.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/szagoruyko/pytorchviz.git (to revision master) to /tmp/pip-req-build-ry20tpiy
  Running command git clone --filter=blob:none --quiet https://github.com/szagoruyko/pytorchviz.git /tmp/pip-req-build-ry20tpiy
  Resolved https://github.com/szagoruyko/pytorchviz.git to commit 0adcd83af8aa7ab36d6afd139cabbd9df598edb7
  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4989 sha256=a9655bd6046527ff566330c6c2770fb601d6f6d82e4dd6d3d040d3f02839d03a
  Stored in directory: /tmp/pip-ephem-wheel-cache-39rhtci0/wheels/2f/dc/0b/2c27f813b826f9b5aaff63702efa3ecd47d144528369111ed0
Successfully built torchviz


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project')
sys.path.insert(0,'/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/projec/code/')
sys.path.insert(0,'/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/code/utils')

In [ ]:
print(sys.path)

['/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/utils', '/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/src', '/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code', '/content', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from drug_ae import AeDrug
from genotype_ae import AeGene
from util import *
import pandas as pd
from torchsummary import summary
tensor_transform = transforms.ToTensor()
import math
from torchviz import make_dot

In [ ]:
drug_dataset = CustomDataset("/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/data/drug2fingerprint.txt")
gene_dataset = CustomDataset("/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/data/cell2mutation.txt")

# DataLoader is used to load the dataset
# for training
drug_loader = torch.utils.data.DataLoader(dataset=drug_dataset,
                                     batch_size=32,
                                     shuffle=True)
gene_loader = torch.utils.data.DataLoader(dataset=gene_dataset,
                                     batch_size=64,
                                     shuffle=True)


In [ ]:
# Training the drug auto encoder and plotting losses
drug_ae_model = AeDrug()
drug_losses = train_model(drug_ae_model,drug_loader,50,"autoencode_drug_epoch_32")
plot(drug_losses,300,"drug_plot")
#make_dot(drug_ae_model(drug_loader), params=dict(list(drug_ae_model.named_parameters()))).render("rnn_torchviz", format="png")
print(drug_ae_model)
dl_df=pd.DataFrame(drug_losses)
dl_df.to_csv("drug_loss.csv")
print("Training Drug component completed")
# Training the cell auto encoder and plotting losses
cell_ae_model = AeGene()
cell_losses = train_model(cell_ae_model,gene_loader,50,"autoencode_cell_epoch_32")
plot(cell_losses,300,"cell_plot")
cl_df=pd.DataFrame(cell_losses)
cl_df.to_csv("cell_loss.csv")
print("Training Drug component completed")

(X_train,Y_train,X_valid,Y_valid,cell2id_mapping,drug2id_mapping) = prepare_train_data("/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/data/drugcell_train.txt","/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/data/drugcell_val.txt","/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/data/cell2ind.txt","/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/data/drug2ind.txt")
drug_fingerprint = pd.read_csv("/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/data/drug2fingerprint.txt",header=None)
numpy_arr = torch.from_numpy(drug_fingerprint.to_numpy().astype(np.float32))

#print(numpy_arr)
cell_mutation =  pd.read_csv("/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/data/cell2mutation.txt",header=None)
cell_arr = torch.from_numpy(cell_mutation.to_numpy().astype(np.float32))
drug_encodings = get_encodings_auto_encoder("/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/output/model/autoencode_drug_epoch_3250",numpy_arr)
cell_encodings = get_encodings_auto_encoder("/content/drive/MyDrive/FA22 MS DS/Semester_2/SP23 MACHINE LEARNING BIOINFORMATCS 4062/Project/project_code/project_code/output/model/autoencode_cell_epoch_3250",cell_arr)



In [ ]:
def get_encoded_data(X,Y,drug_encodings,cell_encodings):
    X = X.detach().numpy()
    y =[]
    print(type(X))
    X_encoded = []
    for i in range(len(X)):
        embedding = []
        numpy_embedding = []
        cell_idx,drug_idx = int(X[i][0].item()),int(X[i][1].item())
        #print(cell_idx,drug_idx)
        embedding.extend(cell_encodings[cell_idx])
        embedding.extend(drug_encodings[drug_idx])
        for k in embedding:
            numpy_embedding.append(k.item())
        X_encoded.append(numpy_embedding)
    for i in range(len(Y)):
        y.append(Y[i].item())
    X_encoded = np.array(X_encoded)
    return X_encoded,y

In [ ]:
x_train_encoded,y_train = get_encoded_data(X_train,Y_train,drug_encodings,cell_encodings)
x_valid_encoded, y_valid = get_encoded_data(X_valid,Y_valid,drug_encodings,cell_encodings)
print(x_train_encoded)
print("-----1------")
print(x_train_encoded.shape)
print("-----2------")
#print(y_train)
print("-----3------")
print(len(y_train),np.array(y_train).shape)
print("-----4------")
#print(x_valid_encoded)
print(x_valid_encoded.shape)
#print(y_valid)
print("-----5------")
print(len(y_valid))

In [ ]:
from sklearn.linear_model import ElasticNet

reg_en = ElasticNet(random_state=0)

reg_en.fit(x_train_encoded,y_train)
y_pred_en=reg_en.predict(x_valid_encoded)
en_preas=pearson_corr(torch.Tensor(y_valid),torch.Tensor(y_pred_en))
print(en_preas)

from sklearn.ensemble import RandomForestRegressor

reg_rf = RandomForestRegressor(bootstrap=False, criterion='squared_error', max_depth=10,
           max_features=20, max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

reg_rf.fit(x_train_encoded,y_train)
y_pred_rf=reg_rf.predict(x_valid_encoded)
rf_preas=pearson_corr(torch.Tensor(y_valid),torch.Tensor(y_pred_rf))
print(rf_preas)



In [ ]:
from sklearn.linear_model import LinearRegression

reg_lin =  LinearRegression()
reg_lin.fit(x_train_encoded,y_train)
y_pred_lin =  reg_lin.predict(x_valid_encoded)

lin_preas=pearson_corr(torch.Tensor(y_valid),torch.Tensor(y_pred_lin))
print(lin_preas)
print(type(y_train),type(y_pred_lin))

from sklearn.neural_network import MLPRegressor
reg_mlp=MLPRegressor(random_state=1, max_iter=500,alpha=0.1,learning_rate='adaptive')
reg_mlp.fit(x_train_encoded,y_train)
y_pred_mlp =  reg_mlp.predict(x_valid_encoded)

mlp_preas=pearson_corr(torch.Tensor(y_valid),torch.Tensor(y_pred_mlp))
print(mlp_preas)
print(type(y_train),type(y_pred_lin))

In [ ]:
from scipy import stats

xy_df_mlp=pd.DataFrame()
xy_df_mlp["Drug"]=(X_valid.T)[1]
xy_df_mlp["y_actual"]=y_valid
xy_df_mlp["y_pred"]=y_pred_mlp
unique_list=list(np.unique(xy_df_mlp['Drug']))
print(len(unique_list))
drugs=[]
pc=[]
for i in unique_list:
  new_df=xy_df_mlp[xy_df_mlp['Drug']==i]
  if len(new_df)>1:
      correlation, pvalue = stats.pearsonr(new_df['y_actual'], new_df['y_pred'])
      if not (math.isnan(correlation) or math.isnan(pvalue)):
          pc.append([correlation, pvalue])
          drugs.append(i)
  #pear_df['Drug']=i

pear_df=pd.DataFrame({"Drug":drugs,"pearson_correlation":np.array(pc).T[0],"pvalue":np.array(pc).T[1]})
'''print(np.mean(pear_df['pearson_correlation']))
print(pear_df)'''
pear_df.to_csv("mlp_pear.csv")

In [ ]:
xy_df_lin=pd.DataFrame()
xy_df_lin["Drug"]=(X_valid.T)[1]
xy_df_lin["y_actual"]=y_valid
xy_df_lin["y_pred"]=y_pred_lin
unique_list_lin=list(np.unique(xy_df_lin['Drug']))
print(len(unique_list_lin))
drugs=[]
pc=[]
for i in unique_list_lin:
  new_df=xy_df_lin[xy_df_lin['Drug']==i]
  if len(new_df)>1:
      correlation, pvalue = stats.pearsonr(new_df['y_actual'], new_df['y_pred'])
      if not (math.isnan(correlation) or math.isnan(pvalue)):
          pc.append([correlation, pvalue])
          drugs.append(i)
  #pear_df['Drug']=i

pear_df_lin=pd.DataFrame({"Drug":drugs,"pearson_correlation":np.array(pc).T[0],"pvalue":np.array(pc).T[1]})
pear_df_lin.to_csv("lin_pear.csv")

In [ ]:
xy_df_rf=pd.DataFrame()
xy_df_rf["Drug"]=(X_valid.T)[1]
xy_df_rf["y_actual"]=y_valid
xy_df_rf["y_pred"]=y_pred_rf
unique_list_rf=list(np.unique(xy_df_rf['Drug']))
print(len(unique_list_rf))
drugs=[]
pc=[]
for i in unique_list_rf:
  new_df=xy_df_rf[xy_df_rf['Drug']==i]
  if len(new_df)>1:
      correlation, pvalue = stats.pearsonr(new_df['y_actual'], new_df['y_pred'])
      if not (math.isnan(correlation) or math.isnan(pvalue)):
          pc.append([correlation, pvalue])
          drugs.append(i)
  #pear_df['Drug']=i

pear_df_rf=pd.DataFrame({"Drug":drugs,"pearson_correlation":np.array(pc).T[0],"pvalue":np.array(pc).T[1]})

pear_df_rf.to_csv("lin_rf.csv")